### 一、本地运行调试代码

网络结构在main.py之后，执行`run main.py`可以本地调试。

执行下面代码可使用快捷键`shift+enter`

In [2]:
run main.py

('结核杆菌目标检测', '数据已经下载到', './data/input/TBDetection', '文件夹中。')
('本地训练提供', '45%', '的数据，线上训练会提供全量数据。')
('数据集详细信息请访问:', '数据来源网络')
download data done...


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/mgy/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

### 二、提交到GPU训练

#### Windows环境执行

通过执行训练命令，`-e=`是整个数据集循环10次，`-b=`是每次训练读取的数据量为32 

In [ ]:
! flyai.exe train -e=20 -b=8

#### Mac和Linux执行

给执行脚本授权

In [6]:
! chmod +x ./flyai

通过执行训练命令，整个数据集循环10次，每次训练读取的数据量为 32 

In [ ]:
! ./flyai train -e=5 -b=1

/bin/bash: /tmp/_MEITVm1Nu/libtinfo.so.5: no version information available (required by /bin/bash)
./linux: /tmp/_MEITVm1Nu/libtinfo.so.5: no version information available (required by ./linux)
./linux: /tmp/_MEITVm1Nu/libtinfo.so.5: no version information available (required by ./linux)
2020-07-21 09:41:31: 代码提交中请等待 ...


### 三、FlyAI的Python环境

#### Windows用户，Python路径在：

In [ ]:
! %USERPROFILE%\.flyai\env\python.exe

#### pip路径：

In [ ]:
! %USERPROFILE%\.flyai\env\Scripts\pip.exe list

#### 如本地遇到No module name 'xxx'问题时可以执行下面代码安装

In [ ]:
! %USERPROFILE%\.flyai\env\Scripts\pip.exe install xxx -i https://pypi.flyai.com/simple 

#### Mac和Linux用户，Python路径在：

In [ ]:
! ~/.flyai/env/bin/python3

#### pip路径：

In [ ]:
! ~/.flyai/env/bin/pip list

#### 如本地遇到No module name 'xxx'问题时可以执行下面代码安装

In [ ]:
! ~/.flyai/env/bin/pip install xxx -i https://pypi.flyai.com/simple

### 四、样例代码说明

##### `app.yaml`

> 是项目的配置文件，项目目录下**必须**存在这个文件，是项目运行的依赖。

##### `processor.py`

> **样例代码中已做简单实现，可供查考。**
>
> 处理数据的输入输出文件，把通过csv文件返回的数据，处理成能让程序识别、训练的矩阵。
>
> 可以自己定义输入输出的方法名，在`app.yaml`中声明即可。
>
> ```python
>     def input_x(self, image_path):
>         '''
>     	参数为csv中作为输入x的一条数据，该方法会被dataset.next_train_batch()
>     	和dataset.next_validation_batch()多次调用。可在该方法中做数据增强
>     	该方法字段与app.yaml中的input:->columns:对应
>     	'''
>         pass
>
>     def output_x(self, image_path):
>          '''
>     	参数为csv中作为输入x的一条数据，该方法会被dataset.next_train_batch()
>     	和dataset.next_validation_batch()多次调用。
>     	该方法字段与app.yaml中的input:->columns:对应
>     	'''
>         pass
>
>     def input_y(self, labels):
>         '''
>         参数为csv中作为输入y的一条数据，该方法会被dataset.next_train_batch()
>     	和dataset.next_validation_batch()多次调用。
>     	该方法字段与app.yaml中的output:->columns:对应
>         '''
>         pass
>
>     def output_y(self, data):
>         '''
>         输出的结果，会被dataset.to_categorys(data)调用
>         :param data: 预测返回的数据
>         :return: 返回预测的标签
>         '''
>         pass
>
> ```

##### `main.py`

> **样例代码中已做简单实现，可供查考。**
>
> 程序入口，编写算法，训练模型的文件。在该文件中实现自己的算法。
>
> 通过`dataset.py`中的`next_batch`方法获取训练和测试数据。
>
> ```python
> '''
> Flyai库中的提供的数据处理方法
> 传入整个数据训练多少轮，每批次批大小
> '''
> dataset = Dataset(epochs=args.EPOCHS, batch=args.BATCH)
> #获取训练数据
> x_train, y_train = dataset.next_train_batch()
> #获取验证数据
> x_val, y_val = dataset.next_validation_batch()
> ```
>
> 通过`model.py`中的`save_model`方法保存模型
>
> ```python
> # 模型操作辅助类
> model = Model(dataset)
> model.save_model(YOU_NET)
> ```
>
> **如果使用`PyTorch`框架，需要在`net.py`文件中实现网络。其它用法同上。**

##### `model.py`

> **样例代码中已做简单实现，可供查考。**
>
> 训练好模型之后可以继承`flyai.model.base`包中的`base`重写下面三个方法实现模型的保存、验证和使用。
>
> ```python
>    def predict(self, **data):
>         '''
>         使用模型
>       	:param data: 模型的输入的一个或多个参数
>         :return:
>         '''
>         pass
>
>     def predict_all(self, datas):
>         '''
>         （必须实现的方法）评估模型，对训练的好的模型进行打分
>       	:param datas: 验证集上的随机数据，类型为list
>         :return outputs: 返回调用模型评估之后的list数据
>         '''
>         pass
>
>     def save_model(self, network, path=MODEL_PATH, name=MODEL_NAME, overwrite=False):
>         '''
>         保存模型
>         :param network: 训练模型的网络
>         :param path: 要保存模型的路径
>         :param name: 要保存模型的名字
>         :param overwrite: 是否覆盖当前模型
>         :return:
>         '''
>         self.check(path, overwrite)
>
> ```

##### `predict.py`

>**样例代码中已做简单实现，可供查考。**
>
>对训练完成的模型使用和预测。

##### `path.py`

> 可以设置数据文件、模型文件的存放路径。

##### `dataset.py`

> 该文件在**FlyAI开源库**的`flyai.dataset`包中，通过`next_train_batch()`和`next_validation_batch()`方法获得`x_train` `y_train` `x_val` `y_val`数据。
>
> FlyAI开源库可以通过`pip3 install -i https://pypi.flyai.com/simple flyai` 安装。

***

### [FlyAI全球人工智能专业开发平台，一站式服务平台](https://flyai.com)

**扫描下方二维码，及时获取FlyAI最新消息，抢先体验最新功能。**



[![GPL LICENSE](https://www.flyai.com/images/coding.png)](https://flyai.com)



